Finding:

Total records:
- vendor A = 10
- vendor B = 10

Missing values in column:
- county = 0
- dob = 5
- ssn = 8
- disposition = 1

Average ingested data latency is 30.4 hours.

Objectives and outcomes:

Allowed values and formats:
Due to the missing value occurance, the data shows:

1. record_value: start with A** or B**
> pass (all records start with either A** or B**)

2. vendor: value should be in (vendor A, vendor B)
> pass (all vendors are either 'vendor a' or 'vendor B')

3. county: counties in the US only
> pass (all counties are in the US)

4. dob: format should be YYYY/MM/DD
> fail (5 out of 20 records)

5. ssn: format should be XXX-XX-XXXX for individual
> fail (8 out of 20 records)

6. disposition: value should be in (Convicted, Dismissed, Pending)
> fail (2 out of 20 records)

7. record_date and ingest_time: dates should be in TimeStamp
> pass

Cross-column logic
1. dates in ingest_time should be later than the dates in record_date.
> pass

2. if disposition is not null, record_date should be not null.
> fail (1 out of 20 records)

3. if disposition is 'pending', record_date should be less than 30 days of ingest_time.
> pass

3. convicted records should have ssn.
> fail (2 out of 20 records)

4. age should be reasonable with under 100 years old.
> fail (5 out of 20 records)

5. date of birth should be earlier than record_date.
> fail (5 out of 20 records)

6. ingest_time should be within SLA at 48 hours from record_date.
> pass

Cross-row logic (same key, different values)
1. record_id should map to one ssn
> fail (18 out of 20 records)

2. same person should not appear twice in the records
> pass

## Vendor A Dataset

In [1]:
%%sql
-- Display vendor A
SELECT * FROM vendor_a

,record_id,vendor,county,dob,ssn,disposition,record_date,ingest_time
0,A001,VendorA,Cook County,1985-04-12,123-45-6789,Convicted,2024-12-28 08:30:00,2024-12-29 10:15:00
1,A002,VendorA,Cook County,1990-09-21,NaN,Dismissed,2024-12-28 09:10:00,2024-12-29 11:00:00
2,A003,VendorA,Los Angeles County,1978-01-05,987-65-4321,Pending,2024-12-27 14:20:00,2024-12-28 18:45:00
3,A004,VendorA,Los Angeles County,1982-06-18,555-22-1111,Convicted,2024-12-27 16:40:00,2024-12-28 17:10:00
4,A005,VendorA,Harris County,1995-11-02,NaN,Convicted,2024-12-26 10:05:00,2024-12-28 12:30:00
5,A006,VendorA,Harris County,1988-03-14,444-88-9999,Dismissed,2024-12-26 13:55:00,2024-12-28 14:05:00
6,A007,VendorA,Cook County,1971-07-09,333-77-6666,Convicted,2024-12-29 08:00:00,2024-12-30 09:30:00
7,A008,VendorA,Maricopa County,1992-12-30,NaN,Pending,2024-12-28 15:10:00,2024-12-29 16:40:00
8,A009,VendorA,Maricopa County,1980-05-22,222-66-5555,Convicted,2024-12-28 17:35:00,2024-12-29 18:50:00
9,A010,VendorA,Los Angeles County,1976-10-01,111-55-4444,Dismissed,2024-12-29 09:25:00,2024-12-30 10:10:00


In [2]:
%%sql
-- Display vendor B
SELECT * FROM vendor_b

,record_id,vendor,county,dob,ssn,disposition,record_date,ingest_time
0,B001,VendorB,Cook County,NaN,123-45-6789,Convicted,2024-12-28 08:30:00,2024-12-28 09:00:00
1,B002,VendorB,Cook County,1990-09-21,NaN,Unknown,2024-12-28 09:10:00,2024-12-28 09:40:00
2,B003,VendorB,Los Angeles County,NaN,987-65-4321,Pending,2024-12-27 14:20:00,2024-12-27 15:00:00
3,B004,VendorB,Los Angeles County,1982-06-18,NaN,Convicted,2024-12-27 16:40:00,2024-12-27 17:05:00
4,B005,VendorB,Harris County,NaN,555-22-1111,NaN,2024-12-26 10:05:00,2024-12-26 10:45:00
5,B006,VendorB,Harris County,1988-03-14,NaN,Dismissed,2024-12-26 13:55:00,2024-12-26 14:10:00
6,B007,VendorB,Cook County,NaN,333-77-6666,Convicted,2024-12-29 08:00:00,2024-12-29 08:30:00
7,B008,VendorB,Maricopa County,1992-12-30,NaN,Pending,2024-12-28 15:10:00,2024-12-28 15:35:00
8,B009,VendorB,Maricopa County,NaN,222-66-5555,Convicted,2024-12-28 17:35:00,2024-12-28 18:00:00
9,B010,VendorB,Los Angeles County,1976-10-01,NaN,Dismissed,2024-12-29 09:25:00,2024-12-29 09:50:00


In [3]:
%%sql
-- Display united-states.csv
SELECT * FROM "united-states"

,FIPS State and County Codes,Geographic area name,County,State
0,1,Alabama,Alabama,Alabama
1,1001,"Autauga County, Alabama",Autauga County,Alabama
2,1003,"Baldwin County, Alabama",Baldwin County,Alabama
3,1005,"Barbour County, Alabama",Barbour County,Alabama
4,1007,"Bibb County, Alabama",Bibb County,Alabama
...,...,...,...,...
3189,56037,"Sweetwater County, Wyoming",Sweetwater County,Wyoming
3190,56039,"Teton County, Wyoming",Teton County,Wyoming
3191,56041,"Uinta County, Wyoming",Uinta County,Wyoming
3192,56043,"Washakie County, Wyoming",Washakie County,Wyoming


In [4]:
%%sql
-- Combine two tables
CREATE VIEW vendor_all AS(
    SELECT * FROM vendor_a
    UNION ALL (SELECT * FROM vendor_b))

,Count


#### Missing Value Flag

In [6]:
%%sql
-- Flag missing values into new columns
CREATE VIEW vendor_mis_value AS (
        SELECT *,
        (DATEDIFF('year', CAST(dob AS DATE), current_date) - CASE WHEN
           (DATE_PART('month', current_date), DATE_PART('day', current_date)) <
           (DATE_PART('month', CAST(dob AS DATE)), DATE_PART('day', CAST(dob AS DATE)))
           THEN 1 ELSE 0 END) AS age,
       (CASE WHEN NULLIF(NULLIF(county,''),'NaN') IS NULL THEN 1 ELSE 0 END) AS mis_county,
       (CASE WHEN NULLIF(NULLIF(dob,''),'NaN') IS NULL THEN 1 ELSE 0 END) AS mis_dob,
       (CASE WHEN NULLIF(NULLIF(ssn,''),'NaN') IS NULL THEN 1 ELSE 0 END) AS mis_ssn,
       (CASE WHEN NULLIF(NULLIF(disposition,''),'NaN') IS NULL THEN 1 ELSE 0 END) as mis_disposition,
       (DATEDIFF('hour', CAST(record_date AS TIMESTAMP), CAST(ingest_time AS TIMESTAMP))) as latency_hour,
       (DATEDIFF('minute', CAST(record_date AS TIMESTAMP), CAST(ingest_time AS TIMESTAMP))) as latency_minute
FROM vendor_all)

CatalogException: Catalog Error: View with name "vendor_mis_value" already exists!

In [125]:
%%sql
SHOW vendor_mis_value
-- drop view vendor_mis_value

,column_name,column_type,null,key,default,extra
0,record_id,VARCHAR,YES,None,None,None
1,vendor,VARCHAR,YES,None,None,None
2,county,VARCHAR,YES,None,None,None
3,dob,VARCHAR,YES,None,None,None
4,ssn,VARCHAR,YES,None,None,None
5,disposition,VARCHAR,YES,None,None,None
6,record_date,VARCHAR,YES,None,None,None
7,ingest_time,VARCHAR,YES,None,None,None
8,age,BIGINT,YES,None,None,None
9,mis_county,INTEGER,YES,None,None,None


In [7]:
%%sql
SELECT * FROM vendor_mis_value

,record_id,vendor,county,dob,ssn,disposition,record_date,ingest_time,age,mis_county,mis_dob,mis_ssn,mis_disposition,latency_hour,latency_minute
0,A001,VendorA,Cook County,1985-04-12,123-45-6789,Convicted,2024-12-28 08:30:00,2024-12-29 10:15:00,40,0,0,0,0,26,1545
1,A002,VendorA,Cook County,1990-09-21,None,Dismissed,2024-12-28 09:10:00,2024-12-29 11:00:00,35,0,0,1,0,26,1550
2,A003,VendorA,Los Angeles County,1978-01-05,987-65-4321,Pending,2024-12-27 14:20:00,2024-12-28 18:45:00,47,0,0,0,0,28,1705
3,A004,VendorA,Los Angeles County,1982-06-18,555-22-1111,Convicted,2024-12-27 16:40:00,2024-12-28 17:10:00,43,0,0,0,0,25,1470
4,A005,VendorA,Harris County,1995-11-02,None,Convicted,2024-12-26 10:05:00,2024-12-28 12:30:00,30,0,0,1,0,50,3025
5,A006,VendorA,Harris County,1988-03-14,444-88-9999,Dismissed,2024-12-26 13:55:00,2024-12-28 14:05:00,37,0,0,0,0,49,2890
6,A007,VendorA,Cook County,1971-07-09,333-77-6666,Convicted,2024-12-29 08:00:00,2024-12-30 09:30:00,54,0,0,0,0,25,1530
7,A008,VendorA,Maricopa County,1992-12-30,None,Pending,2024-12-28 15:10:00,2024-12-29 16:40:00,33,0,0,1,0,25,1530
8,A009,VendorA,Maricopa County,1980-05-22,222-66-5555,Convicted,2024-12-28 17:35:00,2024-12-29 18:50:00,45,0,0,0,0,25,1515
9,A010,VendorA,Los Angeles County,1976-10-01,111-55-4444,Dismissed,2024-12-29 09:25:00,2024-12-30 10:10:00,49,0,0,0,0,25,1485


In [15]:
%%sql
-- Check data quality by count missing values
SELECT vendor, COUNT(*) as count_records,
       SUM(mis_county) AS mis_county,
       SUM(mis_dob) AS mis_dob,
       SUM(mis_ssn) AS mis_ssn,
       SUM(mis_disposition) as mis_disposition,
       AVG(latency_hour) as latency_hour,
       AVG(latency_minute) as latency_minute
 FROM vendor_mis_value
 GROUP BY vendor
 ORDER BY vendor

,vendor,count_records,mis_county,mis_dob,mis_ssn,mis_disposition,latency_hour,latency_minute
0,VendorA,10,0.0,0.0,3.0,0.0,30.4,1824.5
1,VendorB,10,0.0,5.0,5.0,1.0,0.5,28.5


#### Inconsistent Flag:

Assumed business requirements are:

Allowed values and formats
1. record_value: start with A** or B**
2. vendor: value should be in (vendor A, vendor B)
3. county: counties in the US only
4. dob: format should be YYYY/MM/DD
5. ssn: format should be XXX-XX-XXXX for individual
6. disposition: value should be in (Convicted, Dismissed, Pending)
7. record_date and ingest_time: dates should be in TimeStamp

Cross-column logic
1. dates in ingest_time should be later than the dates in 'record_date'.
2. if disposition is not null, record_date should be not null.
3. if disposition is 'pending', record_date should be less than 30 days of ingest_time.
3. convicted records should have ssn.
4. age should be reasonable with under 100 years old.
5. date of birth should be earlier than record_date.
6. ingest_time should be within SLA at 48 hours from record_date.

Cross-row logic
1. record_id should map to one ssn
2. same person should not appear twice in the records

In [88]:
%%sql
-- Inconsistent value and format flag
SELECT vendor,
       -- record_id should start with A** or B**
       SUM(CASE WHEN REGEXP_MATCHES(record_id, '[A-B]*') THEN 0 ELSE 1 END) AS flag_id,
       -- vendor should be in (vendor A, vendor B)
       SUM(CASE WHEN REGEXP_MATCHES(vendor, '[A-B]$') THEN 0 ELSE 1 END) AS flag_vendor,
       -- county should be in the us only
       SUM(CASE WHEN county IN (SELECT county FROM united_states) THEN 0 ELSE 1 END) AS flag_county,
       -- dob format should be YYYY/MM/DD
       SUM(CASE WHEN REGEXP_MATCHES(dob, '[0-9]{4}-[0-9]{2}-[0-9]{2}') THEN 0 ELSE 1 END) AS flag_dob,
       -- ssn format should be XXX-XXX-XXXX
       SUM(CASE WHEN REGEXP_MATCHES(ssn, '([0-9]{3}-[0-9]{2}-[0-9]{4})') THEN 0 ELSE 1 END) AS flag_ssn,
       -- disposition value should be in (Convicted, Dismissed, Pending)
       SUM(CASE WHEN disposition IN ('Convicted', 'Dismissed', 'Pending') THEN 0 ELSE 1 END) AS flag_disp,
       -- record_date and ingest_time: dates should be in TimeStamp
       SUM(CASE WHEN TRY_CAST(record_date AS TIMESTAMP) IS NOT NULL
           AND TRY_CAST(ingest_time AS TIMESTAMP) IS NOT NULL THEN 0 ELSE 1 END) AS flag_date

FROM vendor_mis_value
GROUP BY vendor
ORDER BY vendor

,vendor,flag_id,flag_vendor,flag_county,flag_dob,flag_ssn,flag_disp,flag_date
0,VendorA,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,VendorB,0.0,0.0,0.0,5.0,5.0,2.0,0.0


In [119]:
%%sql
-- Cross Column logic flag
SELECT vendor,
       -- dates in ingest_time should be later than the dates in 'record_date'
       SUM(CASE WHEN record_date < ingest_time THEN 0 ELSE 1 END) AS c_rec_ing_flag,
       -- if disposition is not null, record_date should be not null
       SUM(CASE WHEN disposition IS NOT NULL AND
                     record_date IS NOT NULL THEN 0 ELSE 1 END) AS c_dis_rec_flag,
       -- if disposition is 'pending', record_date should be less than 30 days of ingest_time
       SUM(CASE WHEN disposition = 'Pending' AND
                     DATEDIFF('day', CAST(record_date AS TIMESTAMP),CAST(ingest_time AS TIMESTAMP)) > 30 THEN 1 ELSE 0 END) AS c_pending_30_flag,
       -- convicted records should have ssn
       SUM(CASE WHEN disposition = 'Convicted' AND
                     ssn IS NULL THEN 1 ELSE 0 END) AS c_con_ssn_flag,
       -- age should be reasonable with under 100 years old
       SUM(CASE WHEN age < 100 THEN 0 ELSE 1 END) AS c_age_flag,
       -- date of birth should be earlier than record_date
       SUM(CASE WHEN (DATE_PART('year', CAST(dob AS DATE))) <
              (DATE_PART('year', CAST(record_date AS DATE))) THEN 0 ELSE 1 END) AS c_dob_rec_flag,
       -- ingest_time should be within SLA at 48 hours from record_date
       SUM(CASE WHEN CAST(ingest_time AS DATE) <= CAST(record_date AS DATE) + INTERVAL '48 hours' THEN 0 ELSE 1 END) AS c_sla_48_flag

FROM vendor_mis_value
GROUP BY vendor
ORDER BY vendor

,vendor,c_rec_ing_flag,c_dis_rec_flag,c_pending_30_flag,c_con_ssn_flag,c_age_flag,c_dob_rec_flag,c_sla_48_flag
0,VendorA,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,VendorB,0.0,1.0,0.0,1.0,5.0,5.0,0.0


In [122]:
%%sql
-- Cross-row logic (same key, different values)
-- 1. record_id should map to one ssn
SELECT ssn, COUNT(record_id) as count_id
FROM vendor_mis_value
GROUP BY ssn
HAVING count_id > 1
ORDER BY ssn


,ssn,count_id
0,123-45-6789,2
1,222-66-5555,2
2,333-77-6666,2
3,555-22-1111,2
4,987-65-4321,2
5,None,8


In [8]:
%%sql
-- 2. same person should not appear twice in the records
SELECT record_id, vendor, county, dob, ssn, disposition, record_date, count(*) AS cnt
FROM vendor_mis_value
GROUP BY record_id, vendor, county, dob, ssn, disposition, record_date
HAVING cnt > 1


,record_id,vendor,county,dob,ssn,disposition,record_date,cnt


#### Data Quality Scoring

In [21]:
%%sql
-- Calculate data quality score

WITH data_scoring AS (
SELECT
      vendor,
      COUNT(*) AS total_records,
      -- 15% max. Coverage
      AVG(CASE WHEN dob IS NOT NULL THEN 1 ELSE 0 END) * 0.15 AS dob_completeness,
      -- 35% max. PII Completeness
      AVG(CASE WHEN ssn IS NOT NULL THEN 1 ELSE 0 END) * 0.35 AS ssn_completeness,
      -- 30% max. Disposition Accuracy
      AVG(CASE WHEN disposition IS NOT NULL THEN 1 ELSE 0 END) * 0.3 AS disposition_accuracy,
      -- 20% max. Freshness
      (SUM(CASE WHEN CAST(ingest_time AS TIMESTAMP) <= CAST(record_date AS TIMESTAMP) + INTERVAL '24 hours' THEN 1 ELSE 0 END)*1.0/COUNT(*)) * 0.2 AS freshness_score
      -- Data Quality Score = 35% PII Completeness + 30% Disposition Accuracy + 20% Freshness + 15% Coverage
FROM vendor_mis_value
GROUP BY vendor
ORDER BY vendor)

SELECT (dob_completeness + ssn_completeness + disposition_accuracy + freshness_score) AS data_qual_score, *
FROM data_scoring



,data_qual_score,vendor,total_records,dob_completeness,ssn_completeness,disposition_accuracy,freshness_score
0,0.695,VendorA,10,0.150,0.245,0.30,0.0
1,0.720,VendorB,10,0.075,0.175,0.27,0.2


#### SLA Monitoring and Alert

In [22]:
%%sql
-- SLA data ingestion count

SELECT vendor, count(*) as total_records,
       SUM(CASE WHEN CAST(ingest_time AS TIMESTAMP) <= CAST(record_date AS TIMESTAMP) + INTERVAL '24 hours' THEN 1 ELSE 0 END) AS within_sla,
       SUM(CASE WHEN CAST(ingest_time AS TIMESTAMP) <= CAST(record_date AS TIMESTAMP) + INTERVAL '24 hours' THEN 1 ELSE 0 END) *1.0/COUNT(*) AS freshness_score
FROM vendor_mis_value
GROUP BY vendor
ORDER BY vendor


,vendor,total_records,within_sla,freshness_score
0,VendorA,10,0.0,0.0
1,VendorB,10,10.0,1.0


In [25]:
%%sql
-- SLA breach alert

SELECT vendor, COUNT(*) AS total_records
FROM vendor_mis_value
WHERE CAST(ingest_time AS TIMESTAMP) > CAST(record_date AS TIMESTAMP) + INTERVAL '24 hours'
GROUP BY vendor

,vendor,total_records
0,VendorA,10
